In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from matplotlib import pyplot as plt
from calendar import monthrange
from pandas.tseries.offsets import MonthBegin
import datetime

In [ ]:
path_to_file_customers = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/olist_customers_dataset.csv'
path_to_file_orders = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/olist_orders_dataset.csv'
path_to_file_order_item = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/olist_order_items_dataset.csv'

In [ ]:
customers_data = pd.read_csv(path_to_file_customers , sep = ',')
order_item_data = pd.read_csv(path_to_file_order_item , sep = ',', parse_dates=['shipping_limit_date'])
orders_data = pd.read_csv(path_to_file_orders , sep = ',',parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date','order_estimated_delivery_date'])

In [ ]:
customers_data

In [ ]:
orders_data

In [ ]:
orders_data.order_status.unique ()

In [ ]:
df_merged = pd.merge(
    customers_data, orders_data,
    left_on='customer_id',
    right_on='customer_id'
)

In [ ]:
customer_unique_data = df_merged.query("order_status == 'delivered'") \
    .groupby(['customer_unique_id'], as_index = False) \
    .agg({'customer_id' : 'count'}) \
    .query("customer_id == 1")

В первой таблице содержатся данные о всех заказах, совершенных пользователями. А во второй имеется информация об их статусе. Четкого ответа на вопрос, что считать за "покупку" нет, так нет уточняющей информации для чего необходимо знать количество клиентов, которые совершали покупку только 1 раз. Изначально, мне казалось, что правильным решением будет отбор заказов, которые завершились оплатой. С другой стороны, заказ мог быть отменен клиентом или магазином, и прибыль компании не изменилась (или, например, не выросла конверсия какого-либо рекламного продвижения и т.д.). Поэтому было принято решение отобрать "доставленные" заказы, то есть успешно завершенных заказов, и уже из данного списка отобрать уникальных клиентов, которые не имеют более одного успешно завершенного заказа. Также стоит отметить, что существуют пул незавершенных заказов, которые могут как успешно завершиться, так и отмениться, их мы тоже не будем брать в счёт.

In [ ]:
customer_unique_data.shape[0]

In [ ]:
#2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (10 баллов)

Немного изучив данные становится понятно, что отследить "воронку" движения заказа невозможно, так как при смене статуса заказа, записи не дублируются, а статус изменяется в уже имеющемся номере заказа. По этой причине было принято решение определить "не доставленный заказ", как заказ, который не был и не будет доставлен. То есть стоит убрать из нашего среза статусы со статусами "создан", "подтверждён", "выставлен счёт", "в процессе сборки заказа", "отгружен со склада", так как эти заказы потенциально могут быть доставлены и нет смысла искать в них причины. Также стоит не брать во внимание заказы со статусом "доставлен", так как это успешно завершенная сделка. Остаются 2 типа "недоступен" и "отменён"

In [ ]:
orders_data.query("order_status == 'unavailable'").order_delivered_customer_date.unique()

In [ ]:
orders_data.query("order_status == 'unavailable'").order_delivered_carrier_date.unique()

In [ ]:
#проверка влияния дельты на статус заказа для "недоступно"
orders_data_unavailable = orders_data.query("order_status == 'unavailable'").sort_values('order_purchase_timestamp')
orders_data_unavailable['delta'] = orders_data_unavailable.order_approved_at - orders_data_unavailable.order_purchase_timestamp
orders_data_unavailable.sort_values('delta')

Были проверено, что такие заказы не были доставлены (order_estimated_delivery_date = 'NaT') и не были переданы в службу доставки. Возникла гипотеза влияния количества времени между оформлением заказа и его оплатой на статус. Также возникла гипотеза о зависимости статуса заказа от пользователя (например, отправлен в "черный список"). Обе теории не подтвердились, поэтому данные заказы мы объединим под общей причиной "заказ недоступен"

In [ ]:
#проверка влияния дельты на статус заказа для "досталвено"
orders_data_delivered = orders_data.query("order_status == 'delivered'").sort_values('order_purchase_timestamp')
orders_data_delivered['delta'] = orders_data_delivered.order_approved_at - orders_data_delivered.order_purchase_timestamp
orders_data_delivered.sort_values('delta', ascending = False).head(10)

In [ ]:
# отбираем отмененные заказы
orders_data_canceled = orders_data.query("order_status == 'canceled'")
orders_data_canceled.order_delivered_customer_date.unique()

Заметно, что у нас имеется 6 случаев, когда заказ был доставлен, но имеет статус "отменен". Данные случаи мы также не будем рассматривать, так как они не подходят по нашим условиям (не был и не будет доставлен). Теперь мы имеем 619 случаев того, что заказ не был доставлен по причине отмены, а также 609 случаев "недоступных" заказов.

In [ ]:
orders_data_canceled = orders_data_canceled.query("order_delivered_customer_date == 'NaT'")
orders_data_canceled

In [ ]:
orders_data_canceled_with_item =pd.merge(
    orders_data_canceled, order_item_data,
    left_on='order_id',
    right_on='order_id',
    how='inner'
)
orders_data_canceled_with_item

In [ ]:
orders_data_canceled_with_item['delta'] = orders_data_canceled_with_item.order_estimated_delivery_date - orders_data_canceled_with_item.shipping_limit_date
orders_data_canceled_with_item.sort_values('delta').head(9)

Дальнейший поиск причин упирается только в гипотезы и догадки, так как данных не достаточно, для того, чтобы классифицировать отмененные заказы. Мне удалось найти только 8 записей с аномалиями - крайняя дата доставки продавцом товара в логистическую службу больше крайней даты доставки заказа, что могло повлиять на отмену заказа (а могло и нет). По этой причине было принято решение ограничиться двумя "поверхностными" причинами почему заказ не был доставлен.

In [ ]:
orders_data_canceled['month'] = orders_data_canceled.order_estimated_delivery_date.dt.month
orders_data_canceled['year'] = orders_data_canceled.order_estimated_delivery_date.dt.year
orders_data_unavailable['month'] = orders_data_unavailable.order_estimated_delivery_date.dt.month
orders_data_unavailable['year'] = orders_data_unavailable.order_estimated_delivery_date.dt.year

In [ ]:
orders_data_canceled

In [ ]:
orders_data_unavailable

In [ ]:
#неизвестно
count_unavailable = orders_data_unavailable.groupby(['year','month'], as_index = False) \
    .agg({'order_id' : 'count'}) \
    .groupby(['month'], as_index = False) \
    .agg({'order_id' : 'mean'}) \
    .sort_values('month')
count_unavailable

In [ ]:
#отменено
count_canceled = orders_data_canceled.groupby(['year','month'], as_index = False) \
    .agg({'order_id' : 'count'}) \
    .groupby(['month'], as_index = False) \
    .agg({'order_id' : 'mean'}) \
    .round(1) \
    .sort_values('month')
count_canceled

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

rects1 = ax.bar(count_canceled.month - 0.3/2, count_canceled.order_id, 0.3, label='canceled')
rects2 = ax.bar(count_canceled.month + 0.3/2, count_unavailable.order_id, 0.3, label='gunavailable')
ax.set_title('Среднее количество недоставленных заказов в разбивке по месяцам')
ax.set_xticks(count_canceled.month)
#ax.set_xticklabels(count_canceled.month)
ax.legend()

Было найдено среднее количество заказов, которые не доставили, в разбивке по месяцам (если компании присуща определенная сезонность). А в месяц в среднем количество составит:

In [ ]:
print('Среднее количество заказов, которые не доставили по причине "заказа отменён":', round(count_canceled.order_id.mean(),2))
print('Среднее количество заказов, которые не доставили по причине "неизвестно":', round(count_unavailable.order_id.mean(),2))

In [ ]:
# .
# 3. По каждому товару определить, в какой день недели товар чаще всего покупается. (7 баллов)
#

In [ ]:
order_item_data

In [ ]:
data_product_orders = pd.merge(
    order_item_data, orders_data,
    left_on='order_id',
    right_on='order_id',
    how = 'left'
)

In [ ]:
data_product_orders = data_product_orders.query("order_approved_at != 'NaT'")

В данном случае покупкой будем считать факт оплаты товара, так как, если имеется какая-либо зависимость товара от покупки, то стоит отследить в какой день недели была произведена оплата.

In [ ]:
data_product_orders['week_day'] = data_product_orders.order_approved_at.dt.day_name ()

In [ ]:
data_product_orders = data_product_orders.groupby(['product_id', 'week_day'], as_index = False ) \
    .agg({'order_id' : 'count'}) \
    .sort_values(['product_id', 'order_id'], ascending=[True, False])

In [ ]:
# уникальный айди продукта и его максимальное количество продаж
product_max = data_product_orders.groupby(['product_id'], as_index = False) \
    .agg({'order_id' : 'max'}) \
    .rename(columns = {'order_id' : 'max_order_id'})
product_max

In [ ]:
data_product_orders

In [ ]:
data_product_orders = pd.merge(
    data_product_orders, product_max,
    left_on='product_id',
    right_on='product_id',
    how='left'
)

In [ ]:
data_product_orders = data_product_orders.loc[data_product_orders['order_id'] == data_product_orders['max_order_id']]
del data_product_orders['order_id']

In [ ]:
data_product_orders.pivot(index = 'product_id', columns = 'week_day', values = 'max_order_id').fillna(0)

В данном случае, необходимо было сохранить записи, где несколько максимумов. Для этого был создан отдельный датафрейм, в котором вычислен максимум покупок в разбивке по продукту и номеру недели, а после удалены все значения из исходного датафрейма в группировке по продукту и номеру недели, где число покупок не совпадало с максимумом

In [ ]:
#4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
#Не стоит забывать, что внутри месяца может быть не целое количество недель. 
#Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов)

В данном задании покупка также будет фиксироваться за пользователем, если он совершил оплату, а дата покупки - дата подтверждения оплаты товара

In [ ]:
all_customers = df_merged.query("order_approved_at != 'NaT'")
all_customers = all_customers.assign(weeks_in_month = round(all_customers.order_approved_at.dt.days_in_month / 7,2))
all_customers = all_customers.assign(month = all_customers.order_approved_at.dt.month)
all_customers

In [ ]:
customers_count_in_week = all_customers.groupby(['customer_unique_id', 'month'], as_index = False ) \
    .agg({'order_id' : 'count'}) \
    .sort_values(['customer_unique_id'], ascending=[True]) \
    .rename(columns = {'order_id' : 'count_order_id'})

In [ ]:
#для подсчёта среднего значения количества недель в месяц, с учётом високосного года
count_weeks_in_month = all_customers.groupby(['month'], as_index = False) \
                                    .agg({'weeks_in_month' : 'mean'})
count_weeks_in_month

In [ ]:
customers_count_in_week = pd.merge(
    customers_count_in_week, count_weeks_in_month,
    left_on='month',
    right_on='month',
    how='left'
)
customers_count_in_week

In [ ]:
customers_count_in_week = customers_count_in_week.assign(count_order_in_week = round(customers_count_in_week.count_order_id/customers_count_in_week.weeks_in_month,3))

In [ ]:
customers_count_in_week.pivot(index = 'customer_unique_id', columns = 'month', values = 'count_order_in_week').fillna(0)

In [ ]:
#5.1. Выполните когортный анализ пользователей.

In [ ]:
#5.2. В период с января по декабрь выявите когорту с самым высоким retention на 3-й месяц.

In [ ]:
df_merged = pd.merge(
    customers_data, orders_data,
    left_on='customer_id',
    right_on='customer_id'
)

In [ ]:
df_merged = df_merged.query("order_approved_at != 'NaT'")
start_date = min(df_merged.order_approved_at)
start_date = (start_date + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin()).to_pydatetime().date()

In [ ]:
df_merged = df_merged.assign(date_purchase = (df_merged.order_approved_at + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin()).dt.date)
df_merged = df_merged.assign(delta = ((df_merged.date_purchase - start_date) /  np.timedelta64 ( 1 , 'M')).apply(pd.np.ceil))

In [ ]:
data_first_purchase = df_merged.groupby(['customer_unique_id'], as_index = False).agg({'date_purchase':'min'}).rename(columns = {'date_purchase' : 'first_date_purchase'})
# для подсчёта количества клиентов 
data_first_purchase = data_first_purchase.merge(df_merged.groupby(['customer_unique_id'], as_index = False).agg({'customer_id':'count'}),
    left_on='customer_unique_id',
    right_on='customer_unique_id',
    how = 'left'
)

In [ ]:
df_merged.date_purchase = pd.to_datetime(df_merged.date_purchase)
data_first_purchase.first_date_purchase = pd.to_datetime(data_first_purchase.first_date_purchase)

In [ ]:
# Уникальные временные интервалы
y = df_merged.date_purchase.min().year
m = df_merged.date_purchase.min().month
d = df_merged.date_purchase.min().day
data_date = []
month_year = []
number_month = []
data_date.append((datetime.datetime.strptime(f"{y}-{m}-{d}", '%Y-%m-%d').date()))
number_month.append(f"#1")
month_year.append(f"{m}-{y}")
for i in range(int(df_merged.delta.max())):
    m += 1
    if ((m-1)%12 == 0):
        m = 1
        y = y + 1
    data_date.append((datetime.datetime.strptime(f"{y}-{m}-{d}", '%Y-%m-%d').date()))
    month_year.append(f"{m}-{y}")
    number_month.append(f"#{i+2}")

In [ ]:
retention = np.zeros((len(data_date),len(data_date))) 
# отбираем уникальные айди по первой покупке за каждый месяц, после чего присоединяем по очереди тех клиентов, кто вернулся
# и считаем их количество, подливая в матрицу
for i in range(len(data_date)):
    for j in range(i, len(data_date)):
            retention[i][j-i] = int(pd.merge(data_first_purchase.query(f"first_date_purchase == '{data_date[i]}'") \
                            .groupby(['customer_unique_id'], as_index = False) \
                            .agg({'customer_id':'count'}),
                     df_merged.query(f"date_purchase == '{data_date[j]}'") \
                            .groupby(['customer_unique_id'], as_index = False) \
                            .agg({'customer_id':'count'}),
                     left_on='customer_unique_id',
                     right_on='customer_unique_id',
                     how = 'inner') \
                .agg({'customer_id_y' : 'count'})[0])
            if (i>0):
                retention[i][-i:] = 'NaN'

In [ ]:
#переводим в доли
for i in range(len(data_date)):
    k = retention[i][0]
    for j in range(len(data_date)):
        if (retention[i][0] == 0):
            continue
        else:
            retention[i][j] = round(retention[i][j]/k,2)

In [ ]:
#конвертируем в pandas
data_retention = pd.DataFrame(data=retention, 
                              index=month_year, 
                              columns=number_month)

In [ ]:
data_retention_style = (data_retention
            .style
            .set_caption('Retention - table')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
data_retention_style

Пользователи нашего сервиса в большинстве случаев совершают 1 покупку, и не возвращаются под своим уникальным id

In [ ]:
#6. RFM - сегментация

До расчётов составлящих R, F и M необходимо отметить несколько нюансов. Сегментация будет проводиться только из клиентов, которые получили свои заказы (delivered). Время от последней покупки будет считаться от даты подтверждения заказа (order_approved_at) до последней даты подтвержденного заказа- самого позднего клиента (можно было и посчитать и до текущей даты, но от этого сегментация не изменится, так как будут подобраны аналогичные границы метрик). Количество покупок будет рассчитано как количество чеков (будем объединять несколько товаров в один чек), так как это позволит нам отобрать клиентов с высокой частотой. А если имеются оптовые закупщики, то это можно будет понять по их товарообороту. Сумма покупок будет считаться как сумма по полю "price", так как, в случае, когда покупается несколько товаров одного айди, то записи дублируются (нет необходимости умножать на количество товаров в чеке)

In [ ]:
customers_data = pd.read_csv(path_to_file_customers , sep = ',')
order_item_data = pd.read_csv(path_to_file_order_item , sep = ',', parse_dates=['shipping_limit_date'])
orders_data

In [ ]:
all_data = pd.merge(
    customers_data, orders_data,
    left_on='customer_id',
    right_on='customer_id'
)

In [ ]:
all_data = pd.merge(
    all_data, order_item_data,
    left_on='order_id',
    right_on='order_id'
)
all_data

In [ ]:
r_segment = pd.merge(
    customers_data, orders_data,
    left_on='customer_id',
    right_on='customer_id'
)
r_segment = r_segment.query("order_status == 'delivered' and order_approved_at != 'NaT'")

In [ ]:
#r_segment
last_date = r_segment.groupby(['customer_unique_id'], as_index = False) \
            .agg({'order_approved_at':'max'}).order_approved_at.max() + pd.to_timedelta('1 days 00:00:00')
r_segment = r_segment.groupby(['customer_unique_id'], as_index = False) \
            .agg({'order_approved_at':'max'})
r_segment = r_segment.assign(delta = last_date - r_segment.order_approved_at)
r_segment

In [ ]:
#
f_segment = all_data.query("order_status == 'delivered'") \
                    .groupby(['customer_unique_id'], as_index = False) \
                    .agg({'order_id':'nunique'}) \
                    .rename(columns = {'order_id' : 'count_order'})
f_segment

In [ ]:
m_segment = all_data.query("order_status == 'delivered'") \
    .groupby(['customer_unique_id'], as_index = False) \
    .agg({'price':'sum'}) \
    .rename(columns = {'price' : 'trade_turnover'})
m_segment

In [ ]:
rfm_segment = pd.merge(
    r_segment[['customer_unique_id', 'delta']], 
                        pd.merge(
                            f_segment, m_segment,
                            left_on='customer_unique_id',
                            right_on='customer_unique_id'
                                ),
    left_on='customer_unique_id',
    right_on='customer_unique_id'
)
rfm_segment

In [ ]:
print(rfm_segment.delta.quantile(0))
print(rfm_segment.delta.quantile(0.33))
print(rfm_segment.delta.quantile(0.66))
print(rfm_segment.delta.quantile(0.99))

Всего будет выделено 3 кластера от 1 до 3, где 3 - положительный для компании (больше всего денег, высокая частота или недавняя покупка). Разделение сегмента по дате последней покупки произведено поровну между максимальным большой разницей и минимальной разницей (1/3 на сегмент). Недостаточно данных для определения целевой аудитории сервиса, чтобы определить границы кластеров, исходя из логики. По этой причине, границы были найдены через квантиль

In [ ]:
#r
rfm_segment['R'] = pd.cut(rfm_segment.delta, 
                bins = [pd.to_timedelta('0 days 00:00:00'), 
                        pd.to_timedelta(rfm_segment.delta.quantile(0.33)),
                        pd.to_timedelta(rfm_segment.delta.quantile(0.66)),
                        pd.to_timedelta(rfm_segment.delta.quantile(1))], 
                labels = ['3' , 
                          '2', 
                          '1'])

In [ ]:
rfm_segment.groupby(['count_order'], as_index = False) \
            .agg({'customer_unique_id' : 'count'})

В сегменте по частоте не представляется возможным разбить самый наполненный клиентами сегмент - 1 покупка. Поэтому этот сегмент будет принимать значение "1", как самый популярный. Аналогично отобран второй сегмент - две покупки, третий сегмент - больше двух покупок.

In [ ]:
#f
rfm_segment['F'] = pd.cut(rfm_segment.count_order, 
                bins = [0, 
                        1,
                        2,
                        200], 
                labels = ['1' , 
                          '2', 
                          '3'])

In [ ]:
#m
rfm_segment['M'] = pd.cut(rfm_segment.trade_turnover, 
                bins = [0.0, 
                        rfm_segment.trade_turnover.quantile(0.5),
                        rfm_segment.trade_turnover.quantile(0.8),
                        rfm_segment.trade_turnover.quantile(1)], 
                labels = ['1' , 
                          '2', 
                          '3'])

Сегмент "M" был разделён по аналогу с ABC-классификатором. Худшая половина пользователей - это "1", лучшие 20% - "3"

In [ ]:
rfm_segment.drop(['delta','count_order','trade_turnover'], axis= 1 , inplace= True)

In [ ]:
rfm_segment.R = rfm_segment.R.astype('int32')
rfm_segment.F = rfm_segment.F.astype('int32')
rfm_segment.M = rfm_segment.M.astype('int32')

In [ ]:
rfm_segment